In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

In [5]:
from ipynb.fs.full.my_metrics import precision_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


### Задание 1. Подбор оптимальных гиперпараметров для ALS

- Попробуйте улучшить базовый вариант ALS, изменяя следующие параметры
  - regularization, iterations
  - factors
  - Вес (TF_IDF, BM25  взвешивание)
  
- Посчитайте метрики (Precision@5, MAP@5) для разных наборов гиперпараметров и выберете лучший набор

In [6]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [7]:
# Заведем фиктивный item_id (если юзер покупал товары не из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

/Users/annasmejoukha/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [16]:
factors = [32, 64, 96, 128, 256]
regularization = [0.1, 0.05, 0.01, 0.005, 0.001]
iterations = [5, 10, 15, 20, 25]

In [17]:
result_dict = {
    'factors': [],
    'regularization': [],
    'iterations': [],
    'precision_at_k': [],
    'precision_tfidf': [],
    'precision_bm25': []
}

In [19]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)]
    return res

In [20]:
def get_precision(df, user_item_matrix, factor=64, reg=0.01, iteration=15):
    user_item_matrix_tfidf = tfidf_weight(user_item_matrix.T).T
    user_item_matrix_bm25 = bm25_weight(user_item_matrix.T).T
    
    als_model = AlternatingLeastSquares(factors=factor, 
                                            regularization=reg, 
                                            iterations=iteration,
                                            num_threads=0)
    als_model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=False)
    df['als'] = df['user_id'].apply(lambda x: get_recommendations(x, model=als_model, N=5))
    
    tfidf_model = AlternatingLeastSquares(factors=factor, 
                                            regularization=reg, 
                                            iterations=iteration,
                                            num_threads=0)
    tfidf_model.fit(csr_matrix(user_item_matrix_tfidf).T.tocsr(), show_progress=False)
    df['tfidf'] = df['user_id'].apply(lambda x: get_recommendations(x, model=tfidf_model, N=5))
    
    bm25_model = AlternatingLeastSquares(factors=factor, 
                                            regularization=reg, 
                                            iterations=iteration,
                                            num_threads=0)
    bm25_model.fit(csr_matrix(user_item_matrix_bm25).T.tocsr(), show_progress=False)
    df['bm25'] = df['user_id'].apply(lambda x: get_recommendations(x, model=bm25_model, N=5))
    
    precision_dict = {}
    for name_col in df.columns[2:]:
        precision_dict[name_col] = round(df.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(), 4)
        
    return precision_dict

In [21]:
import os
os.environ['MKL_NUM_THREADS'] = '1'

In [22]:
%%time
from itertools import product

for factor, reg, iteration in product(factors, regularization, iterations):
    res_df = result[['user_id', 'actual']]
    precision = get_precision(res_df, user_item_matrix, factor, reg, iteration)

    result_dict['factors'].append(factor)
    result_dict['regularization'].append(reg)
    result_dict['iterations'].append(iteration)
    result_dict['precision_at_k'].append(precision['als'])
    result_dict['precision_tfidf'].append(precision['tfidf'])
    result_dict['precision_bm25'].append(precision['bm25'])

CPU times: user 16h 9min 55s, sys: 3h 14min 5s, total: 19h 24min 1s
Wall time: 3h 31min 5s


In [23]:
import json

In [24]:
with open('./my_precision.json', 'w') as f:
    json.dump(result_dict, f, indent=4)

In [25]:
with open('./my_precision.json') as f:
    result_dict = json.load(f)

In [26]:
# датафрейм из получившихся значений
df_res = pd.DataFrame(result_dict)

In [27]:
df_res.head(5)

,factors,regularization,iterations,precision_at_k,precision_tfidf,precision_bm25
0,32,0.1,5,0.1912,0.2030,0.1952
1,32,0.1,10,0.1874,0.1980,0.1924
2,32,0.1,15,0.1881,0.2004,0.1901
3,32,0.1,20,0.1868,0.1981,0.1903
4,32,0.1,25,0.1834,0.2002,0.1881


In [29]:
precision_idx = np.argmax(df_res.precision_at_k)
precision_tfidf_idx = np.argmax(df_res.precision_tfidf)
precision_bm_25_idx = np.argmax(df_res.precision_bm25)

In [31]:
print(f'лучший результат без взвешивания: {df_res.iloc[precision_idx, 3]}\nПараметры:\n{df_res.iloc[precision_idx, :3]}')

лучший результат без взвешивания: 0.2002
Параметры:
factors           96.00
regularization     0.05
iterations         5.00
Name: 55, dtype: float64


In [33]:
print(f'лучший результат с TF-IDF: {df_res.iloc[precision_tfidf_idx, 4]}\nПараметры:\n{df_res.iloc[precision_tfidf_idx, :3]}')

лучший результат с TF-IDF: 0.203
Параметры:
factors           32.0
regularization     0.1
iterations         5.0
Name: 0, dtype: float64


In [34]:
print(f'лучший результат с взвешиванием BM25: {df_res.iloc[precision_bm_25_idx, 5]}\nПараметры:\n{df_res.iloc[precision_bm_25_idx, :3]}')

лучший результат с взвешиванием BM25: 0.2402
Параметры:
factors           256.0
regularization      0.1
iterations          5.0
Name: 100, dtype: float64
